In [1]:
import xarray as xr
import numpy as np
from scipy.interpolate import NearestNDInterpolator

In [2]:
# open ice shelf mask
ds_isf=xr.open_dataset('Mask_Iceshelf_IMBIE2_v2.nc')

# get variable needed for interpolation
isflon1d=ds_isf.lon.values.flatten()
isflat1d=ds_isf.lat.values.flatten()
isfid1d=-ds_isf.Iceshelf_extrap.values.flatten() # the - is for compatibility with cdfisf

# open NEMO coordinates
# get NEMO ice shelf mask
fNEMOmask='eORCA025.L121-OPM0262_mesh_mask.nc'
ds_NEMO=xr.open_dataset(fNEMOmask)

# get variables needed for interpolation
lonNEMO=ds_NEMO.glamt.values.squeeze()
latNEMO=ds_NEMO.gphit.values.squeeze()

In [3]:
# nearest neigbourg interpolation
# is cdo.remapnn faster ? The default python take 2m for eORCA025 => could be a limiting factor for eORCA36 or higher 
interp = NearestNDInterpolator(list(zip(isflon1d, isflat1d)), isfid1d)
isfNEMO = interp(lonNEMO, latNEMO)

In [4]:
# create the data set
ds_isfNEMO=ds_isf.drop_vars(["Iceshelf","Iceshelf_extrap"]).drop_dims(["x","y"])
ds_isfNEMO.coords["nav_lon"] = (("y","x"), lonNEMO)
ds_isfNEMO.coords["nav_lat"] = (("y","x"), latNEMO)
ds_isfNEMO["Iceshelf_extrap"]=xr.DataArray(
    data=isfNEMO,
    dims=["y", "x"],
)

In [5]:
# retreive ice shelf ID map
# get id mask
ds_isfNEMO['mask_isf']=xr.DataArray(
    data=ds_isfNEMO["Iceshelf_extrap"]*ds_NEMO['tmaskutil'].squeeze().values[:,:]*(1-ds_NEMO['tmask'].squeeze().values[0,:,:]),
    dims=["y", "x"],
    attrs=dict(unit='[]',
               long_name='ice shelf ID mask')
)

In [6]:
# Define NEMO coastal mask of the area to parametrised
def get_coastal_msk(mask_in,lewp):
    nj_out=mask_in.shape[0]
    ni_out=mask_in.shape[1]

    if lewp:
        mask=np.zeros(shape=(nj_out,ni_out+2))
        mask[:,1:-1]=mask_in
        mask[:, 0]=mask[:,-2]
        mask[:,-1]=mask[:, 1]
        xslc=slice(0,ni_out)
    else:
        mask=mask_in
        xslc=slice(1,-1)
        
    mask_coast=np.zeros(shape=(nj_out,ni_out))
    mask_coast[1:-1,xslc]= ( mask[1:-1,1:-1] + 
                             mask[0:-2,1:-1] + mask[2::,1:-1] + mask[1:-1,0:-2] + mask[1:-1,2::] +
                             mask[0:-2,0:-2] + mask[2::,2::]  + mask[2::,0:-2]  + mask[0:-2,2::] ) * mask[1:-1,1:-1]
    mask_coast[(mask_coast > 1) & (mask_coast < 9)] = 10
    mask_coast[mask_coast!=10]=np.nan
    mask_coast=mask_coast.astype(np.float32)
    mask_coast[mask_coast==10]=1
    mask_coast[np.isnan(mask_coast)]=0
    
    return mask_coast

In [7]:
# retreive ice shelf front from ice shelf defined in Rignot
front_msk=get_coastal_msk(np.where(ds_isfNEMO['mask_isf'].values!=0,0,1),True)
front_msk=front_msk * ds_NEMO['tmask'].squeeze().values[0,:,:] * ds_isfNEMO["Iceshelf_extrap"].squeeze().values

ds_isfNEMO["mask_isf_front"]=xr.DataArray(
    data=front_msk,
    dims=["y", "x"],
    attrs=dict(unit='[]',
            long_name='ice shelf front ID mask')
)

In [8]:
# write global attributes
ds_isfNEMO.attrs=dict(Source='IMBIE2 (Mask_Iceshelf_IMBIE2_v2.nc)',
                      Method='Nearest neighbourg',
                      Compatibility='The masked variables are compatible with {}'.format(fNEMOmask),
                      Creator='Pierre Mathiot (9/10/2024)')

In [9]:
# write the dataset
ds_isfNEMO.to_netcdf('isfmsk.nc')

In [10]:
# compute ice shelf text file for CDFTOOLS
nisf=0
for idx,name in enumerate(ds_isf.NAME.values):
    if ((-ds_isf.ID.values[idx] in np.unique(ds_isfNEMO['mask_isf'].values)) and (name != '') and (ds_isf.MeltRignot.values[idx] != 0.0)):
        isfd=ds_NEMO.isfdraft.where(ds_isfNEMO.mask_isf.values == -ds_isf.ID.values[idx])
        ctxt='{0:4d} {1:} {2:5d} {3:5d} {4:6.1f}'.format(-ds_isf.ID.values[idx],name[0:4].upper(),np.min(isfd).values.astype(int),np.max(isfd).values.astype(int),ds_isf.MeltRignot.values[idx])
        print(ctxt)
        nisf=nisf+1
print(nisf)

  -1 JELB    91   582   -1.0
  -2 FIMB    76   626   23.5
  -4 VIGR   234   469    3.2
  -5 NIVL    58   403    3.9
  -7 LAZA    99   403    6.3
  -8 BORC    67   535    7.5
  -9 BAUD    71   506   14.1
 -11 PRIN    62   387   -2.0
 -13 SHIR   134   289    5.7
 -15 RAYN   143   202    6.7
 -17 WILM    64   322   10.0
 -18 EDWA   100   125    4.2
 -21 AMER    99  1991   35.5
 -22 PUBL   137   403    1.5
 -24 WEST    62   569   27.2
 -26 SHAC    99   670   72.6
 -27 TRAC    42   131    3.0
 -29 CONG    52   119    3.6
 -31 VINC   126   453    5.0
 -33 TOTT   231  1423   63.2
 -34 MOSC   294  1160   27.4
 -38 HOLM    85   486    6.7
 -41 DIBB   407   702    8.1
 -43 MERT   148   636    7.9
 -45 NINN   293   653    2.2
 -47 COOK   190   586    4.6
 -53 RENN    79   561    7.0
 -55 LILL    53   619    3.4
 -59 MARI    29   309    2.4
 -61 AVIA   149   202    1.4
 -63 NANS    10   347    1.1
 -64 DRYG   146   755    7.6
 -66 ROSS    10  1020   49.1
 -67 ROSS    58   972   -1.4
 -74 SWIN    9